In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
# We pull these directly from the 'tf' object to avoid version conflicts
Sequential = tf.keras.models.Sequential
TextVectorization = tf.keras.layers.TextVectorization
Embedding = tf.keras.layers.Embedding
GlobalAveragePooling1D = tf.keras.layers.GlobalAveragePooling1D
Dense = tf.keras.layers.Dense

print(f"TensorFlow is ready: {tf.__version__}")


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# This converts the Series into a clean NumPy array
train_labels = train_df['label'].map({'ham': 0, 'spam': 1}).values.astype('float32')
test_labels = test_df['label'].map({'ham': 0, 'spam': 1}).values.astype('float32')

In [ ]:
MAX_WORDS = 1000
MAX_LEN = 100

vectorizer = TextVectorization(
    max_tokens=MAX_WORDS,
    output_sequence_length=MAX_LEN
)
vectorizer.adapt(train_df['message'].values)

# 3. Create the Model
model = tf.keras.Sequential([
    vectorizer,
    tf.keras.layers.Embedding(1000, 16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Train - We pass the text as a numpy array to be safe
model.fit(train_df['message'].values, train_labels, epochs=10, verbose=1)



In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # 1. Convert text to a NumPy array with dtype=object
    # This 'locks' the data type so TensorFlow doesn't throw the str768 error
    text_input = np.array([pred_text], dtype=object)

    # 2. Call the model directly
    # Using model(text_input) is often more stable than model.predict in newer TF versions
    prediction = model(text_input, training=False)

    # 3. Extract the probability scalar
    prob = prediction.numpy()[0][0]

    # 4. Determine label
    label = "spam" if prob >= 0.5 else "ham"

    return [float(prob), label]

# Test it
print(predict_message("how are you doing today?"))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
